# Nelder-Mead optimization 

## Luca Sagresti

## 15/12/2019

In [1]:
%clear
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import least_squares

%matplotlib inline

mpl.rcParams['figure.dpi']=100
mpl.rcParams['figure.titlesize']=20
mpl.rcParams['axes.facecolor']='white'        
mpl.rcParams['lines.linewidth']=2.0
mpl.rcParams['axes.linewidth']=2.0
mpl.rcParams['xtick.major.pad']=6
mpl.rcParams['ytick.major.pad']=6
mpl.rcParams['xtick.labelsize']=14
mpl.rcParams['ytick.labelsize']=14
mpl.rcParams['axes.titlesize']=18
mpl.rcParams['axes.labelsize']=18
mpl.rcParams['axes.grid']='True'
mpl.rcParams['axes.axisbelow']='line'
mpl.rcParams['legend.fontsize']=12

## Import Experimental Data

In [4]:
import pandas as pd

folder="~/PHD/NOTEBOOK/Data_exp_Kv/"

# Activation dataset

dataset_act = pd.read_csv(folder+"Act_datasets.csv", skiprows=1)

act_data = dataset_act.to_numpy()

act_data_WT = act_data[:,1]

act_data_S390N = act_data[:,3]

# Inactivation dataset

dataset_inact = pd.read_csv(folder+"Inact_datasets.csv", skiprows=1)

inact_data = dataset_inact.to_numpy()

inact_data_WT = inact_data[:,1]

inact_data_S390N = inact_data[:,3]

# Recovery dataset

dataset_rec = pd.read_csv(folder+"Rec_datasets.csv", skiprows=1)

rec_data = dataset_rec.to_numpy()

rec_data_WT = rec_data[:,1]

rec_data_S390N = rec_data[:,3]



## Activation Part

In [70]:
def ode_Lin_model_Act (S, t, p):
    
    C0=S[0]
    C1=S[1]
    C2=S[2]
    C3=S[3]
    C4=S[4]
    I0=S[5]
    I1=S[6]
    I2=S[7]
    I3=S[8]
    I4=S[9]
    O =S[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 0.5:
        V = -90.0 # mV
        
    if 0.5 <= t < 0.55:
        V = p[11] # Vtest
        
    if 0.55 <= t <= 1.05:
        V = -50.0 # mV
        
    if t > 1.05 :
        V = -50.0 #mV
 


    # wild type parameters
#    alpha_0 = 450.0 #s^-1
#    alpha_1 = 0.23 #s^-1
    
#    beta_0 = 2.0 #s^-1
#    beta_1 = 2.2 #s^-1
    
#    k_CO_0 = 160.0 #s^-1
#    k_CO_1 = 0.27 #s^-1
    
#    k_OC_0 = 245.0 #s^-1
#    k_OC_1 = 0.33 #s^-1
    
#    k_CI = 25.0 #s^-1
#    k_IC = 0.3 #s^-1
    
#    f = 0.37

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [71]:
def Act_Protocol(max_V, DeltaV):
    Vhold = -90.0 #mV
    
    Vtest = np.linspace(Vhold,max_V,np.abs(np.int((max_V-Vhold)/DeltaV))+1)
    
    return Vtest #array of testing voltages

In [72]:
def Boltz_eq(x, V1_2, k):
    return 1.0/(1.0 + np.exp(-(x-V1_2)/k))

In [73]:
def Act_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2     # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Act_Protocol(Vmax, increment)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 1.05
    
    ttest = 0.50 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(Vtest)))

    max_conductance = np.zeros(len(Vtest)) 

    
    for i in range (0,len(Vtest)):
        gamma = np.append(teta, Vtest[i])
        
        f = lambda S,t: ode_Lin_model_Act(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*ttest):,10])
    
    
    g_gmax = (max_conductance-np.amin(max_conductance))/(np.amax(max_conductance)-np.amin(max_conductance))
    
    return g_gmax

In [74]:
def residual_Act(p):
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Act_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_act = -22.70 #mV
    
    exp_k_act = 13.00 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data = Boltz_eq(Vtest,exp_Vhemi_act,exp_k_act)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 1.05
    
    ttest = 0.56 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    sq_err = np.sum(np.subtract(exp_data,Act_LS_func(t,p))**2)
    
    act_cost_func = (1.0/len(Vtest)) * sq_err
    
    return act_cost_func

#### Debugger Activation

In [15]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')
result = minimize(residual_Act, guess, method='Nelder-Mead')
print result

/usr/local/lib/python2.7/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:88: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in double_scalars


 final_simplex: (array([[2.99391284e+02, 1.32513171e-01, 1.26165439e+02, 1.26608444e-01,
        1.03896203e+03, 7.06040899e-01, 2.98805374e+00, 8.18382469e-03,
        3.68351636e+01, 4.08872663e-01, 9.56049627e-09],
       [2.99391311e+02, 1.32513301e-01, 1.26165446e+02, 1.26608281e-01,
        1.03896196e+03, 7.06041070e-01, 2.98805521e+00, 8.18359310e-03,
        3.68351503e+01, 4.08872666e-01, 9.54326344e-09],
       [2.99391374e+02, 1.32513267e-01, 1.26165447e+02, 1.26608423e-01,
        1.03896198e+03, 7.06041007e-01, 2.98805235e+00, 8.18361732e-03,
        3.68351486e+01, 4.08872670e-01, 9.57552283e-09],
       [2.99391339e+02, 1.32513274e-01, 1.26165447e+02, 1.26608347e-01,
        1.03896198e+03, 7.06041031e-01, 2.98805252e+00, 8.18361224e-03,
        3.68351495e+01, 4.08872670e-01, 9.57491393e-09],
       [2.99391303e+02, 1.32513056e-01, 1.26165435e+02, 1.26608698e-01,
        1.03896208e+03, 7.06040746e-01, 2.98805075e+00, 8.18399569e-03,
        3.68351708e+01, 4.08872662e

## Inactivation Part

In [75]:
def ode_Lin_model_WT_inac(C, t, p):
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=p[11]
        
    if 2.00 <= t <= 3.00:
        V=60.0
        
    if t > 3.00 :
        V=60.0
 


    # wild type parameters
#    alpha_0 = 450.0 #s^-1
#    alpha_1 = 0.23 #s^-1
    
#    beta_0 = 2.0 #s^-1
#    beta_1 = 2.2 #s^-1
    
#    k_CO_0 = 160.0 #s^-1
#    k_CO_1 = 0.27 #s^-1
    
#    k_OC_0 = 245.0 #s^-1
#    k_OC_1 = 0.33 #s^-1
    
#    k_CI = 25.0 #s^-1
#    k_IC = 0.3 #s^-1
    
#    f = 0.37

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [76]:
def Inact_Protocol(max_V, DeltaV):
    Vhold = -90.0 #mV
    
    Vtest = np.linspace(Vhold,max_V,np.abs(np.int((max_V-Vhold)/DeltaV))+1)
    
    return Vtest #array of testing voltages

In [77]:
def Boltz_eq_in(x, V1_2, k):
    return 1.0/(1.0 + np.exp((x-V1_2)/k))

In [78]:
def Inact_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2     # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Inact_Protocol(Vmax, increment)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 3.00
    
    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(Vtest)))

    max_conductance = np.zeros(len(Vtest)) 

    
    for i in range (0,len(Vtest)):
        gamma = np.append(teta, Vtest[i])
        
        f = lambda S,t: ode_Lin_model_WT_inac(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*ttest):,10]-r[np.int(Points_per_sec*ttest)-10,10])
    
    
    g_gmax = (max_conductance-np.amin(max_conductance))/(np.amax(max_conductance)-np.amin(max_conductance))
    
    return g_gmax

In [79]:
def residual_Inact(p):
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Inact_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_act = -49.60 #mV
    
    exp_k_act = 5.10 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data = Boltz_eq_in(Vtest,exp_Vhemi_act,exp_k_act)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 3.00
    
    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    sq_err = np.sum(np.subtract(exp_data,Inact_LS_func(t,p))**2)
    
    Inact_cost_func = (1.0/len(Vtest)) * sq_err
    
    return Inact_cost_func

#### Debugger inactivation

In [22]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')

result = minimize(residual_Inact, guess, method='Nelder-Mead')

print (result)

 final_simplex: (array([[ 1.87283017e+03, -2.14379107e-02,  6.98043991e+00,
         4.12054013e+00,  1.58271377e+02, -1.21927350e-01,
         1.58591576e+02,  2.68873337e-02,  1.71431623e+02,
         1.23725250e-04,  5.35995576e-01],
       [ 1.87283019e+03, -2.14379107e-02,  6.98043999e+00,
         4.12054013e+00,  1.58271376e+02, -1.21927351e-01,
         1.58591576e+02,  2.68873331e-02,  1.71431622e+02,
         1.23725216e-04,  5.35995575e-01],
       [ 1.87283019e+03, -2.14379142e-02,  6.98043964e+00,
         4.12054014e+00,  1.58271373e+02, -1.21927358e-01,
         1.58591577e+02,  2.68873290e-02,  1.71431624e+02,
         1.23724866e-04,  5.35995581e-01],
       [ 1.87283014e+03, -2.14379037e-02,  6.98044006e+00,
         4.12054012e+00,  1.58271387e+02, -1.21927332e-01,
         1.58591572e+02,  2.68873438e-02,  1.71431619e+02,
         1.23725277e-04,  5.35995569e-01],
       [ 1.87283020e+03, -2.14379136e-02,  6.98043983e+00,
         4.12054013e+00,  1.58271372e+02, -1

## Recovery Part

In [80]:
def ode_rec_mut (C, t, p):
    
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=60.0
        
    if 2.00 <= t <= (2.00+p[11]):
        V=-90.0
        
    if (2.00+p[11]) < t <= (p[11]+3.00):
        V=60.0
 
    if t > (p[11]+3.00):
        V=-90.0

    # S390N parameters

#    alpha_0 = 450.0 #s^-1
#    alpha_1 = 0.23 #s^-1
    
#    beta_0 = 2.0 #s^-1
#    beta_1 = 2.2 #s^-1
    
#    k_CO_0 = 160.0 #s^-1
#    k_CO_1 = 0.27 #s^-1
    
#    k_OC_0 = 245.0 #s^-1
#    k_OC_1 = 0.33 #s^-1
    
#    k_CI = 25.0 #s^-1
#    k_IC = 0.3 #s^-1
    
#    f = 0.37

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [81]:
def Rec_Protocol(max_t, Deltat):
    min_t = 0.015 #mV
    
    t_pulse = np.linspace(min_t,max_t,np.abs(np.int((max_t-min_t)/Deltat))+1)
    
    return t_pulse #array of testing prepulses

In [82]:
def Simple_exp_eq(x,tau):
    return 1.0 - np.exp(-x / tau)

In [83]:
def Rec_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2     # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    tmax = 0.300 #s
    
    increment = 0.015 #s
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    # Time discretiztion
    tini = 0.0 #s
    
    tend = 4.00 #s
    
    #ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(t_pulse)))

    max_conductance = np.zeros(len(t_pulse)) 

    
    for i in range (0,len(t_pulse)):
        gamma = np.append(teta, t_pulse[i])
        
        f = lambda S,t: ode_Lin_model_WT_rec(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*t_pulse[i]):,10])
    
    
    g_gmax = (max_conductance)/(np.amax(max_conductance))
    
    return g_gmax

In [84]:
def residual_Rec(p):
    
    #define activation sequence protocol
    
    tmax = 300.0 #ms
    
    increment = 15.0 #ms
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_tau = 47.0 #ms
    
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data = Simple_exp_eq(t_pulse,exp_tau)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 4.00
    
#    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    sq_err = np.sum(np.subtract(exp_data,Rec_LS_func(t,p))**2)
    
    Rec_cost_func = (1.0/len(t_pulse)) * sq_err
    
    return Rec_cost_func

#### Debugger recovery

In [28]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')

result = minimize(residual_Rec, guess, method='Nelder-Mead')

print (result)

 final_simplex: (array([[1.32898659e+03, 2.24893687e-01, 5.36490016e+01, 1.99038233e+00,
        3.38452686e+02, 9.27290811e-01, 1.17300914e+02, 3.63562042e-01,
        1.69720986e-01, 9.70914148e-03, 1.36847256e-01],
       [1.32898659e+03, 2.24893706e-01, 5.36489922e+01, 1.99038215e+00,
        3.38452765e+02, 9.27290917e-01, 1.17300922e+02, 3.63562052e-01,
        1.69721171e-01, 9.70913911e-03, 1.36847183e-01],
       [1.32898660e+03, 2.24893652e-01, 5.36490185e+01, 1.99038253e+00,
        3.38452609e+02, 9.27290649e-01, 1.17300890e+02, 3.63562028e-01,
        1.69721082e-01, 9.70914626e-03, 1.36847367e-01],
       [1.32898657e+03, 2.24893685e-01, 5.36490033e+01, 1.99038247e+00,
        3.38452632e+02, 9.27290829e-01, 1.17300925e+02, 3.63562044e-01,
        1.69720688e-01, 9.70913400e-03, 1.36847252e-01],
       [1.32898657e+03, 2.24893695e-01, 5.36489976e+01, 1.99038234e+00,
        3.38452688e+02, 9.27290855e-01, 1.17300925e+02, 3.63562048e-01,
        1.69720917e-01, 9.70913671e

## Launch the main

In [85]:
def residual_Tot(p):

    # Time discretiztion
    tini = 0.0
    
    tend = 4.00
    
#    ttest = 0.56 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)

####################### START OF THE 3 VOLTAGE PROTOCOLS ###################   

    # DEFINE ACTIVATION SEQUENCE PROTOCOL
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest_act = Act_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_act = -22.70 #mV
    
    exp_k_act = 13.00 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data_act = Boltz_eq(Vtest_act,exp_Vhemi_act,exp_k_act)
    
    # here data from perfect fit
    #sq_err_act = np.sum(np.subtract(exp_data_act,Act_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    sq_err_act = np.sum(np.subtract(act_data_WT,Act_LS_func(t,p))**2)
    
    act_cost_func = (1.0/len(Vtest_act)) * sq_err_act
    
    
    # DEFINE INACTIVATION SEQUENCE PROTOCOL
    
#    Vmax = 60.0 #mV
    
#    increment = 10.0 #mV
    
    Vtest_inact = Inact_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_inact = -49.60 #mV
    
    exp_k_inact = 5.10 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data_inact = Boltz_eq_in(Vtest_inact,exp_Vhemi_inact,exp_k_inact)
    
    # here data from perfect fit
    #sq_err_inact = np.sum(np.subtract(exp_data_inact,Inact_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    sq_err_inact = np.sum(np.subtract(inact_data_WT,Inact_LS_func(t,p))**2)
    
    inact_cost_func = (1.0/len(Vtest_inact)) * sq_err_inact
   
   
    # DEFINE RECOVERY SEQUENCE PROTOCOL
    
    tmax = 300.0 #ms
    
    increment = 15.0 #ms
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    #experimental data for recovery time
    
    exp_tau = 47.0 #ms
    
    #evaluation of experimental points trough the declared exponential fit
    
    exp_data_rec = Simple_exp_eq(t_pulse,exp_tau)
    
    # here data from perfect fit
    #sq_err_rec = np.sum(np.subtract(exp_data_rec,Rec_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    sq_err_rec = np.sum(np.subtract(rec_data_WT,Rec_LS_func(t,p))**2)
    
    rec_cost_func = (1.0/len(t_pulse)) * sq_err_rec
    
    
    # sum of the three protocols cost function
    return (act_cost_func + inact_cost_func + rec_cost_func)

In [86]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')

result = minimize(residual_Tot, guess, method='Nelder-Mead')

print (result)

 final_simplex: (array([[6.90867363e+02, 2.53571030e-01, 4.53203391e+01, 2.22005732e+00,
        3.26155969e+02, 3.68993986e-01, 6.58894642e+01, 2.79998440e-01,
        6.49983938e+01, 2.10002528e-01, 3.32001198e-01],
       [6.90867363e+02, 2.53571030e-01, 4.53203391e+01, 2.22005732e+00,
        3.26155969e+02, 3.68993986e-01, 6.58894642e+01, 2.79998440e-01,
        6.49983938e+01, 2.10002528e-01, 3.32001198e-01],
       [6.90867363e+02, 2.53571030e-01, 4.53203391e+01, 2.22005732e+00,
        3.26155969e+02, 3.68993986e-01, 6.58894642e+01, 2.79998440e-01,
        6.49983938e+01, 2.10002528e-01, 3.32001198e-01],
       [6.90867363e+02, 2.53571030e-01, 4.53203391e+01, 2.22005732e+00,
        3.26155969e+02, 3.68993986e-01, 6.58894642e+01, 2.79998440e-01,
        6.49983938e+01, 2.10002528e-01, 3.32001198e-01],
       [6.90867363e+02, 2.53571030e-01, 4.53203391e+01, 2.22005732e+00,
        3.26155969e+02, 3.68993986e-01, 6.58894642e+01, 2.79998440e-01,
        6.49983938e+01, 2.10002528e

## Avoid Local Minima with random kick

In [ ]:
from random import seed
from random import randint
from datetime import datetime


seed(np.int(datetime.now().strftime("%f")))

tmp = np.zeros(len(result_trf.x))

old_guess = result_trf.x

new_guess = np.zeros(len(result_trf.x))

epsilon = 100.0

while(epsilon > 1.0):

    for i in range(0,len(result_trf.x)):
    
        tmp[i] = old_guess[i] + ((-1.0)**(randint(0,99))) * result_trf.x[i] / 10.0
    

    results = least_squares(residual_Act, tmp, bounds=boundaries, method='trf',tr_solver='lsmr')  

    new_guess = results.x
    
    epsilon = np.sum(np.abs(np.subtract(new_guess,old_guess)))
    
    print new_guess
    
    print "The epsilon is"
    print epsilon
    
    new_guess = old_guess

    

[4.06687258e+02 2.79376399e-01 1.83547624e+00 2.47501012e+00
 1.74999788e+02 2.73629542e-01 2.67838255e+02 3.40104914e-01
 2.74352665e+01 2.59762566e-01 3.22855816e-01]
The epsilon is
88.51302071014707
[4.07333640e+02 2.79859221e-01 2.24969408e+00 2.03431508e+00
 1.42211450e+02 2.19966258e-01 2.18819601e+02 3.41088177e-01
 2.74171302e+01 3.30563701e-01 3.89483646e-01]
The epsilon is
89.15829457056691
[4.97092416e+02 2.65500738e-01 1.78994832e+00 2.58024139e+00
 1.75922976e+02 2.87777624e-01 2.67914563e+02 3.49970909e-01
 2.67720517e+01 3.30758664e-01 3.93551462e-01]
The epsilon is
89.0493361497073
[4.05404416e+02 3.15001002e-01 2.15779963e+00 2.13816073e+00
 1.44592021e+02 2.88628602e-01 2.69023507e+02 3.14640323e-01
 2.31808385e+01 2.84115539e-01 2.90074063e-01]
The epsilon is
88.68214563064207
[5.00401776e+02 3.08648588e-01 2.28199576e+00 1.96589297e+00
 1.77062975e+02 2.34619998e-01 2.68248832e+02 2.91170670e-01
 2.69225873e+01 2.70499040e-01 3.60251834e-01]
The epsilon is
93.874341